Image Classification MLOps using Amazon Sagemaker
This notebook lists all the steps that you need to complete the complete this project. 

In [6]:

!pip install smdebug

  Using cached smdebug-1.0.34-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached pyinstrument_cext-0.2.4-cp312-cp312-linux_x86_64.whl
Using cached smdebug-1.0.34-py2.py3-none-any.whl (280 kB)
Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [smdebug]m3/4 [smdebug]]


In [15]:
import sagemaker
import os, time, json
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter
from sagemaker.inputs import TrainingInput
from sagemaker.debugger import ProfilerRule, FrameworkProfile, ProfilerConfig, rule_configs, DebuggerHookConfig, CollectionConfig

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print("Role:", role)
print("Default S3 bucket:", sess.default_bucket())

Role: arn:aws:iam::106660882488:role/service-role/AmazonSageMaker-ExecutionRole-20251027T142948
Default S3 bucket: sagemaker-us-east-1-106660882488


## Dataset
This project uses the Dog Breed Classification dataset provided in the Udacity classroom. The dataset contains images from 133 different dog breeds, covering a wide range of sizes, coat types, and geographic origins. The dataset is already split into training, validation, and testing sets, which supports a clean and reproducible ML workflow. Images vary in lighting, pose, and background, making the classification task more realistic and challenging. This variety encourages strong generalization and helps evaluate the effectiveness of transfer learning when adapting a pre trained model like ResNet to a multi class image classification problem.

In [9]:
# Command to download and unzip data
# Uncomment and run the below two lines of code only the first time when you want to download and upload the data to s3
#!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
#!unzip dogImages.zip


# Upload Data to S3
# Run this cell only the first time, to upload the data once.
LOCAL_DIR = 'dogImages'
S3_BUCKET = sess.default_bucket()
DATA_PREFIX = "dogimages"
input_data_path = sess.upload_data(path=LOCAL_DIR, bucket=S3_BUCKET, key_prefix=DATA_PREFIX)

print(f"input data path: {input_data_path}")

input data path: s3://sagemaker-us-east-1-106660882488/dogimages


In [17]:

# Set Input and Output path on S3 for the project
train = f"s3://{S3_BUCKET}/{DATA_PREFIX}/train"
valid   = f"s3://{S3_BUCKET}/{DATA_PREFIX}/valid"
test = f"s3://{S3_BUCKET}/{DATA_PREFIX}/valid"

timestamp = time.strftime("%Y%m%d-%H%M%S")
output_path = f"s3://{S3_BUCKET}/{DATA_PREFIX}/outputs/{timestamp}"
code_location = f"s3://{S3_BUCKET}/{DATA_PREFIX}/code/{timestamp}"

print(f"output path: {output_path}")
print(f"code location: {code_location}")



output path: s3://sagemaker-us-east-1-106660882488/dogimages/outputs/20251030-000051
code location: s3://sagemaker-us-east-1-106660882488/dogimages/code/20251030-000051


## Hyperparameter Tuning
This section focuses on fine-tuning a pretrained ResNet-50 using SageMaker Hyperparameter Optimization (HPO).
The goal is to systematically explore parameter combinations that improve validation performance.
I use hpo.py as the training entry script so SageMaker can run multiple jobs in parallel with different settings.
Key hyperparameters tuned include learning rate, batch size, and epochs.
Learning rate controls convergence speed, batch size affects stability and generalization, and epochs balance training time versus overfitting.
I chose these ranges—learning rate (1e-4 to 1e-2), batch size (8–32), and epochs (3–10)—to stay within GPU memory and runtime limits.
The objective metric for HPO is validation loss (val_loss), since it measures generalization without leaking test data.
SageMaker automatically tracks printed metrics (val_loss, val_accuracy, test_loss, test_accuracy) from the training script.
All training artifacts and logs are stored in versioned S3 paths to ensure full reproducibility.
After tuning completes, the best model and its optimal hyperparameters are retrieved for final evaluation on the test set.

In [23]:
#Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(1e-4, 1e-2),  
    "batch_size": IntegerParameter(8, 32),             
    "epochs": IntegerParameter(3, 10),                 
}

metric_definitions = [
    {"Name": "val_loss",       "Regex": r"val_loss=([0-9.+-eE]+);"},
    {"Name": "test_loss",      "Regex": r"test_loss=([0-9.+-eE]+);"},
    {"Name": "test_accuracy",  "Regex": r"test_accuracy=([0-9.+-eE]+);"},
    {"Name": "train_loss",     "Regex": r"train_loss=([0-9.+-eE]+);"},
]

objective_metric_name = "val_loss"
objective_type = "Minimize"

In [26]:
# Create estimators for your HPs

INSTANCE_TYPE = "ml.g4dn.xlarge"  

estimator = PyTorch(
    entry_point="hpo.py",
    source_dir=".",
    role=role,
    framework_version="1.13",
    py_version="py39",
    instance_type=INSTANCE_TYPE,
    instance_count=1,
    output_path=output_path,         
    code_location=code_location,     
    metric_definitions=metric_definitions,
    hyperparameters={
        "num_classes": 133, # Dataset consists of 133 classes
        "image_size": 224, # Input requirement for the pre trained ResNet-50 model
        "device": "cuda",            
    },
)

tuner = HyperparameterTuner(
    estimator=estimator,
    metric_definitions=metric_definitions,
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    objective_type=objective_type,
    max_jobs=8,            
    max_parallel_jobs=2,   
)

In [27]:
inputs = {
    "training": TrainingInput(
        s3_data=input_data_path,      
        distribution="FullyReplicated"
    )
}

# Launch HPO tuner
tuner.fit(inputs, wait=True)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.....................................................................................................................................*


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 }                                                                                           │
│    7                                                                                             │
│    8 # Launch HPO tuner                                                                          │
│ ❱  9 tuner.fit(inputs, wait=True)                                                                │
│   10                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/tuner.py:1039 in fit                           │
│                                                                                                  │
│   1036 │   │   │   self._fit_with_estimator_dict(inputs, job_name, include_cls_metadata, estima  │
│   1037 │   │                                                                                     │
│   1038 │   │   if wait:                                                                          │
│ ❱ 1039 │   │   │   self.latest_tuning_job.wait()                                                 │
│   1040 │                                                                                         │
│   1041 │   def _fit_with_estimator(self, inputs, job_name, include_cls_metadata, **kwargs):      │
│   1042 │   │   """Start tuning for tuner instances that have the ``estimator`` field set."""     │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/tuner.py:2347 in wait                          │
│                                                                                                  │
│   2344 │                                                                                         │
│   2345 │   def wait(self):                                                                       │
│   2346 │   │   """Placeholder docstring."""                                                      │
│ ❱ 2347 │   │   self.sagemaker_session.wait_for_tuning_job(self.name)                             │
│   2348                                                                                           │
│   2349                                                                                           │
│   2350 def create_identical_dataset_and_algorithm_tuner(                                         │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/session.p

In [ ]:
# Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()

# Get the hyperparameters of the best trained model
print("Best training job name:", best_estimator.latest_training_job.name)
print("\nBest hyperparameters:")
print(best_estimator.hyperparameters())


## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()